In [ ]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096
dtype = None
load_in_4bit = True


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 256,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,
    loftq_config = None,
)

In [ ]:
from datasets import load_dataset

dataset = load_dataset('ibm/finqa')
print(dataset)

In [ ]:
prompt = """You are an expert in the financial domain tasked with solving a given financial problem. Follow these steps:

1. Read the question carefully.
2. Provide the formula for the given question.
3. Use the formula to caculate the final answer.


### Pretext:
{}

### Posttext:
{}

### Table:
{}

### Question:
{}

### Answer:
{}

### Program:
{}

### GoldInds:
{}

### Output:
{}"""
prompt2 = """You are an expert in the financial domain tasked with solving a given financial problem. Follow these steps:

1. Read the question carefully.
2. Provide the formula for the given question.
3. Use the formula to caculate the final answer.


### Pretext:
{}

### Posttext:
{}

### Table:
{}

### Question:
{}

### Output:
{}"""

EOS_TOKEN = tokenizer.eos_token 
def formatting_prompts_func(examples):
    pre_text = examples["pre_text"]
    post_text       = examples["post_text"]
    table = examples["table"]
    question = examples["question"]
    answer = examples["answer"]
    program_re      = examples["program_re"]
    gold_inds = examples["gold_inds"]
    output      = examples["answer"]
    texts = []
    for a,b,c,d,e,f,g,h in zip(pre_text,post_text, table, question, answer, program_re, gold_inds, output):
        text = prompt2.format(a,b,c,d,h) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }




In [ ]:

train_dataset = dataset['train'].map(formatting_prompts_func, batched = True,)

In [ ]:
train_dataset['text'][0]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

training_args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 400,
        num_train_epochs = 1,
        max_steps = 1000,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.02,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        max_grad_norm=1.0
    )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False,
    args = training_args
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
model.save_pretrained("model1")
tokenizer.save_pretrained("model1")

In [ ]:
import shutil
from google.colab import files
shutil.make_archive("model1", 'zip', "model1")
files.download("model1.zip")


In [ ]:
test_dataset = dataset['test']

In [ ]:
!unzip model1.zip -d /content/model1


In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "model1",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)


In [ ]:
print(len(test_dataset))

In [ ]:
final_response = []
for i in range(len(test_dataset)):
  FastLanguageModel.for_inference(model)
  pre_text = test_dataset[i]["pre_text"]
  post_text = test_dataset[i]["post_text"]
  table = test_dataset[i]["table"]
  question = test_dataset[i]["question"]
  answer = ""
  program_re = ""
  gold_inds = ""
  input_prompt = prompt.format(
          pre_text,
          post_text,
          table,
          question,
          # answer,
          # program_re,
          # gold_inds,
          "",
      )
  inputs = tokenizer(
  [
      input_prompt
  ], return_tensors = "pt").to("cuda")

  input_shape = inputs['input_ids'].shape
  input_token_len = input_shape[1]
  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

  response = tokenizer.batch_decode([outputs[0][input_token_len:]], skip_special_tokens=True)
  final_response.append(response[0])
  if i%100 == 0:
    print(i)

In [ ]:
count = 0
for i in range(len(test_dataset)):
  answer = test_dataset[i]["answer"]
  if final_response[i] == answer:
    count = count + 1
print(count)

In [ ]:
print(count/len(test_dataset)*100)